In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE HACK_WH;
USE DATABASE TOURISM_DB;

CREATE SCHEMA IF NOT EXISTS TOURISM_DB.RAW;
CREATE SCHEMA IF NOT EXISTS TOURISM_DB.CURATED;



In [ ]:
-- (1) Create the dedicated Cortex role
CREATE ROLE IF NOT EXISTS CORTEX_APP_ROLE COMMENT = 'Role for Smart Trip & Safety Concierge Analyst';

-- (2) Grant it usage on key objects
GRANT USAGE ON WAREHOUSE HACK_WH TO ROLE CORTEX_APP_ROLE;
GRANT USAGE ON DATABASE TOURISM_DB TO ROLE CORTEX_APP_ROLE;
GRANT USAGE ON SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;
GRANT USAGE ON SCHEMA TOURISM_DB.META TO ROLE CORTEX_APP_ROLE;

-- (3) Allow data reading and stored procedure calls
GRANT SELECT ON ALL TABLES IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;
GRANT SELECT ON FUTURE TABLES IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;

GRANT SELECT ON ALL VIEWS IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;
GRANT SELECT ON FUTURE VIEWS IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;

-- (4) Optional but recommended: allow viewing stage/file data
GRANT USAGE ON ALL STAGES IN SCHEMA TOURISM_DB.RAW TO ROLE CORTEX_APP_ROLE;

-- (5) Assign to your user so you can test
GRANT ROLE CORTEX_APP_ROLE TO USER JBATACAN;
-- Example: GRANT ROLE CORTEX_APP_ROLE TO USER JOHN_BATACAN;


In [ ]:
CREATE OR REPLACE FILE FORMAT TOURISM_DB.-- (1) Create the dedicated Cortex role
CREATE ROLE IF NOT EXISTS CORTEX_APP_ROLE COMMENT = 'Role for Smart Trip & Safety Concierge Analyst';

-- (2) Grant it usage on key objects
GRANT USAGE ON WAREHOUSE HACK_WH TO ROLE CORTEX_APP_ROLE;
GRANT USAGE ON DATABASE TOURISM_DB TO ROLE CORTEX_APP_ROLE;
GRANT USAGE ON SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;
GRANT USAGE ON SCHEMA TOURISM_DB.META TO ROLE CORTEX_APP_ROLE;

-- (3) Allow data reading and stored procedure calls
GRANT SELECT ON ALL TABLES IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;
GRANT SELECT ON FUTURE TABLES IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;

GRANT SELECT ON ALL VIEWS IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;
GRANT SELECT ON FUTURE VIEWS IN SCHEMA TOURISM_DB.CURATED TO ROLE CORTEX_APP_ROLE;

GRANT EXECUTE ON ALL PROCEDURES IN SCHEMA TOURISM_DB.META TO ROLE CORTEX_APP_ROLE;
GRANT EXECUTE ON FUTURE PROCEDURES IN SCHEMA TOURISM_DB.META TO ROLE CORTEX_APP_ROLE;

-- (4) Optional but recommended: allow viewing stage/file data
GRANT USAGE ON ALL STAGES IN SCHEMA TOURISM_DB.RAW TO ROLE CORTEX_APP_ROLE;

-- (5) Assign to your user so you can test
GRANT ROLE CORTEX_APP_ROLE TO USER <your_username>;
-- Example: GRANT ROLE CORTEX_APP_ROLE TO USER JOHN_BATACAN;
RAW.CSV_STD 
  TYPE=CSV SKIP_HEADER=1 FIELD_OPTIONALLY_ENCLOSED_BY='"';

CREATE STAGE IF NOT EXISTS TOURISM_DB.RAW.STAGE_WEATHER;
CREATE STAGE IF NOT EXISTS TOURISM_DB.RAW.STAGE_POI;
CREATE STAGE IF NOT EXISTS TOURISM_DB.RAW.STAGE_HOLIDAYS;
CREATE STAGE IF NOT EXISTS TOURISM_DB.RAW.STAGE_CRIME;
CREATE STAGE IF NOT EXISTS TOURISM_DB.RAW.STAGE_GTFS;
CREATE STAGE IF NOT EXISTS TOURISM_DB.RAW.STAGE_TRAFFIC;
CREATE STAGE IF NOT EXISTS TOURISM_DB.RAW.STAGE_HOTELS;


In [ ]:
CREATE OR REPLACE TABLE TOURISM_DB.RAW.WEATHER_HOURLY(
  AS_OF_TS TIMESTAMP_TZ, LAT NUMBER(9,6), LON NUMBER(9,6),
  FORECAST_TS TIMESTAMP_TZ, TEMP_C FLOAT, PRECIP_PROB FLOAT, WIND_KPH FLOAT, SRC VARIANT
);

CREATE OR REPLACE TABLE TOURISM_DB.RAW.POI_OPENTRIPMAP(
  FETCHED_TS TIMESTAMP_TZ, POI_ID STRING, NAME STRING, KINDS STRING, OSM_ID STRING,
  LAT FLOAT, LON FLOAT, RATE FLOAT, WIKIDATA STRING, RAW VARIANT
);

CREATE OR REPLACE TABLE TOURISM_DB.RAW.PH_HOLIDAYS(
  FETCHED_TS TIMESTAMP_TZ, DATE DATE, LOCAL_NAME STRING, NAME STRING,
  FIXED BOOLEAN, GLOBAL BOOLEAN, COUNTIES ARRAY, TYPES ARRAY, RAW VARIANT
);

CREATE OR REPLACE TABLE TOURISM_DB.RAW.CRIME_INCIDENTS(
  SRC_FILE STRING, INCIDENT_ID STRING, INCIDENT_TS TIMESTAMP_TZ,
  CATEGORY STRING, CITY STRING, BARANGAY STRING, LAT FLOAT, LON FLOAT, RAW VARIANT
);

CREATE OR REPLACE TABLE TOURISM_DB.RAW.TRAFFIC_SPEEDS(
  SEGMENT_ID STRING, ROAD_NAME STRING, START_LAT FLOAT, START_LON FLOAT,
  END_LAT FLOAT, END_LON FLOAT, AS_OF_TS TIMESTAMP_TZ,
  SPEED_KPH FLOAT, CONGESTION_LEVEL STRING
);

CREATE OR REPLACE TABLE TOURISM_DB.RAW.TRAFFIC_INCIDENTS(
  INCIDENT_ID STRING, SEGMENT_ID STRING, ROAD_NAME STRING,
  LAT FLOAT, LON FLOAT, AS_OF_TS TIMESTAMP_TZ, TYPE STRING, SEVERITY STRING
);

CREATE OR REPLACE TABLE TOURISM_DB.RAW.GTFS_STOPS_EXP(
  STOP_ID STRING, STOP_NAME STRING, STOP_LAT FLOAT, STOP_LON FLOAT, PARENT_STATION STRING, ZONE_ID STRING, RAW VARIANT
);
CREATE OR REPLACE TABLE TOURISM_DB.RAW.GTFS_TRIPS_EXP(
  ROUTE_ID STRING, SERVICE_ID STRING, TRIP_ID STRING, TRIP_HEADSIGN STRING, DIRECTION_ID STRING, RAW VARIANT
);
CREATE OR REPLACE TABLE TOURISM_DB.RAW.GTFS_STOP_TIMES_EXP(
  TRIP_ID STRING, ARRIVAL_TIME STRING, DEPARTURE_TIME STRING, STOP_ID STRING, STOP_SEQUENCE INT, RAW VARIANT
);

CREATE OR REPLACE TABLE TOURISM_DB.RAW.HOTELS(
  HOTEL_ID STRING, NAME STRING, LAT FLOAT, LON FLOAT, CITY STRING, AREA STRING,
  STARS INT, PRICE_NIGHT_PHP NUMBER(10,2), REFUNDABLE BOOLEAN, RATING FLOAT, AMENITIES STRING, SAFETY_SCORE FLOAT
);


In [ ]:
COPY INTO TOURISM_DB.RAW.WEATHER_HOURLY
FROM @TOURISM_DB.RAW.STAGE_WEATHER/weather_hourly.csv
FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.POI_OPENTRIPMAP
FROM @TOURISM_DB.RAW.STAGE_POI/poi_opentripmap.csv
FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.PH_HOLIDAYS
FROM @TOURISM_DB.RAW.STAGE_HOLIDAYS/ph_holidays_2025_2026.csv
FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.CRIME_INCIDENTS
FROM @TOURISM_DB.RAW.STAGE_CRIME/crime_incidents.csv
FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.TRAFFIC_SPEEDS
FROM @TOURISM_DB.RAW.STAGE_TRAFFIC/traffic_speeds.csv
FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.TRAFFIC_INCIDENTS
FROM @TOURISM_DB.RAW.STAGE_TRAFFIC/traffic_incidents.csv
FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.GTFS_STOPS_EXP
  FROM @TOURISM_DB.RAW.STAGE_GTFS/gtfs_stops_expanded.txt
  FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.GTFS_TRIPS_EXP
  FROM @TOURISM_DB.RAW.STAGE_GTFS/gtfs_trips_expanded.txt
  FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.GTFS_STOP_TIMES_EXP
  FROM @TOURISM_DB.RAW.STAGE_GTFS/gtfs_stop_times_expanded.txt
  FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

COPY INTO TOURISM_DB.RAW.HOTELS
FROM @TOURISM_DB.RAW.STAGE_HOTELS/hotels.csv
FILE_FORMAT=(FORMAT_NAME=TOURISM_DB.RAW.CSV_STD);

In [ ]:
CREATE OR REPLACE VIEW TOURISM_DB.CURATED.WEATHER_SUITABILITY AS
SELECT
  LAT,
  LON,
  FORECAST_TS,
  TEMP_C,
  PRECIP_PROB,
  WIND_KPH,
  CASE 
    WHEN LOWER((PARSE_JSON(SRC):city)::STRING) = 'intramuros' THEN 'Manila'
    ELSE COALESCE((PARSE_JSON(SRC):city)::STRING, 'Unknown')
  END AS CITY,
  
  LEAST(
    1.0,
    GREATEST(
      0.0,
      1 - (PRECIP_PROB / 100) * 0.7 - (WIND_KPH / 40) * 0.3
    )
  ) AS SUITABILITY,

  CASE 
    WHEN PRECIP_PROB >= 60 THEN 'Better indoor' 
    ELSE 'Good for outdoor' 
  END AS RECO

FROM TOURISM_DB.RAW.WEATHER_HOURLY;



CREATE OR REPLACE VIEW TOURISM_DB.CURATED.POI AS
SELECT POI_ID, NAME, KINDS, LAT, LON, RATE,
       (COALESCE(RATE,0)) AS POPULARITY_SCORE
FROM TOURISM_DB.RAW.POI_OPENTRIPMAP;

CREATE OR REPLACE VIEW TOURISM_DB.CURATED.HOLIDAYS AS
SELECT DATE, LOCAL_NAME, NAME FROM TOURISM_DB.RAW.PH_HOLIDAYS;

CREATE OR REPLACE VIEW TOURISM_DB.CURATED.SAFETY_INDEX AS
SELECT CITY, BARANGAY, DATE_TRUNC('day', INCIDENT_TS) AS D, COUNT(*) AS INCIDENTS_DAY
FROM TOURISM_DB.RAW.CRIME_INCIDENTS
GROUP BY 1,2,3;

CREATE OR REPLACE VIEW TOURISM_DB.CURATED.TRAFFIC_LATEST AS
SELECT t.*
FROM TOURISM_DB.RAW.TRAFFIC_SPEEDS t
QUALIFY ROW_NUMBER() OVER (PARTITION BY SEGMENT_ID ORDER BY AS_OF_TS DESC)=1;

CREATE OR REPLACE VIEW TOURISM_DB.CURATED.TRAFFIC_ALERTS AS
SELECT SEGMENT_ID, ROAD_NAME, AS_OF_TS, TYPE, SEVERITY, LAT, LON
FROM TOURISM_DB.RAW.TRAFFIC_INCIDENTS
WHERE AS_OF_TS >= DATEADD(day,-7,CURRENT_TIMESTAMP());

CREATE OR REPLACE VIEW TOURISM_DB.CURATED.TRANSIT_NEXT_DEPARTURES AS
WITH BASE AS (
  SELECT st.TRIP_ID, st.STOP_ID, st.ARRIVAL_TIME,
         TO_TIME(ARRIVAL_TIME) AS ARR_T, s.STOP_NAME, s.STOP_LAT, s.STOP_LON, t.ROUTE_ID, t.TRIP_HEADSIGN
  FROM TOURISM_DB.RAW.GTFS_STOP_TIMES_EXP st
  JOIN TOURISM_DB.RAW.GTFS_TRIPS_EXP t  USING (TRIP_ID)
  JOIN TOURISM_DB.RAW.GTFS_STOPS_EXP s ON s.STOP_ID=st.STOP_ID
)
SELECT *
FROM BASE
WHERE ARR_T BETWEEN TIME_FROM_PARTS(07,00,00) AND TIME_FROM_PARTS(10,00,00)
ORDER BY ARR_T;


CREATE OR REPLACE VIEW TOURISM_DB.CURATED.HOTEL_RECO AS
SELECT
  HOTEL_ID, NAME, CITY, AREA, STARS, PRICE_NIGHT_PHP, REFUNDABLE, RATING, AMENITIES, SAFETY_SCORE, LAT, LON,
  (RATING*0.5 + STARS*0.2 + SAFETY_SCORE*0.3) AS SCORE
FROM TOURISM_DB.RAW.HOTELS;


In [ ]:
SELECT DISTINCT(CITY) FROM CURATED.WEATHER_SUITABILITY;

In [ ]:
SELECT * FROM CURATED.POI WHERE KINDS ILIKE '%museum%' LIMIT 10;

In [ ]:
SELECT * FROM CURATED.HOLIDAYS ORDER BY DATE LIMIT 5;

In [ ]:
SELECT CITY,BARANGAY,MAX(INCIDENTS_DAY) AS peak FROM CURATED.SAFETY_INDEX GROUP BY 1,2 ORDER BY peak DESC LIMIT 10;

In [ ]:
SELECT SEGMENT_ID, ROAD_NAME, SPEED_KPH, CONGESTION_LEVEL FROM CURATED.TRAFFIC_LATEST ORDER BY SPEED_KPH ASC LIMIT 5;

In [ ]:
SELECT * FROM CURATED.TRANSIT_NEXT_DEPARTURES LIMIT 10;

In [ ]:
SELECT NAME, PRICE_NIGHT_PHP, SCORE FROM CURATED.HOTEL_RECO ORDER BY SCORE DESC LIMIT 5;

In [ ]:
-- Top museums near Intramuros with current traffic & safety note
SELECT p.NAME, p.LAT, p.LON, p.POPULARITY_SCORE,
       COALESCE(t.CONGESTION_LEVEL,'N/A') AS traffic,
       (SELECT MAX(INCIDENTS_DAY) FROM CURATED.SAFETY_INDEX s 
         WHERE s.CITY='Manila' AND s.BARANGAY IN ('Intramuros','Ermita')) AS safety_recent
FROM CURATED.POI p
LEFT JOIN CURATED.TRAFFIC_LATEST t
  ON t.ROAD_NAME ILIKE '%Roxas%'  -- demo join (or use nearest segment by geodistance in app layer)
WHERE p.KINDS ILIKE '%museum%'
ORDER BY p.POPULARITY_SCORE DESC;

In [ ]:
SELECT FORECAST_TS, TEMP_C, PRECIP_PROB, RECO
FROM CURATED.WEATHER_SUITABILITY
WHERE LAT BETWEEN 14.58 AND 14.60 AND LON BETWEEN 120.97 AND 120.99   -- Intramuros box
  AND FORECAST_TS::DATE=DATE('2025-10-15')
ORDER BY FORECAST_TS;

In [ ]:
SELECT BARANGAY, SUM(INCIDENTS_DAY) AS last_7d
FROM CURATED.SAFETY_INDEX
WHERE D >= DATEADD(day,-7,CURRENT_DATE())
GROUP BY BARANGAY
ORDER BY last_7d DESC;

In [ ]:
SELECT NAME, KINDS, LAT, LON, POPULARITY_SCORE
FROM CURATED.POI
WHERE KINDS ILIKE '%museum%'
ORDER BY POPULARITY_SCORE DESC
LIMIT 5;

In [ ]:
SELECT NAME, PRICE_NIGHT_PHP, REFUNDABLE, SCORE
FROM CURATED.HOTEL_RECO
WHERE CITY IN ('Makati','Taguig','Manila')
ORDER BY SCORE DESC, PRICE_NIGHT_PHP ASC
LIMIT 5;

In [ ]:
SELECT NAME, LAT, LON, RATE, POPULARITY_SCORE
FROM CURATED.POI
LIMIT 5;

In [ ]:
-- Next departures around 8–10 AM (demo)
SELECT ROUTE_ID, TRIP_ID, TRIP_HEADSIGN, STOP_NAME, ARR_T
FROM CURATED.TRANSIT_NEXT_DEPARTURES
LIMIT 10;

-- Current worst congestion (to avoid)
SELECT ROAD_NAME, SPEED_KPH, CONGESTION_LEVEL
FROM CURATED.TRAFFIC_LATEST
ORDER BY SPEED_KPH ASC
LIMIT 5;


In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE TOURISM_DB;
USE SCHEMA CURATED;
CREATE STAGE IF NOT EXISTS SEMANTIC_UPLOAD_STAGE;


In [ ]:
CREATE OR REPLACE VIEW TOURISM_DB.CURATED.WEATHER_BY_CITY AS
SELECT
  CITY_NAME AS CITY,
  DATE(FORECAST_TS) AS FORECAST_DATE,
  PRECIP_PROB,
  SUITABILITY,
  RECO
FROM TOURISM_DB.CURATED.WEATHER_SUITABILITY;
